In [35]:
from Plan import CreateSamples_renewed
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib

In [36]:
importlib.reload(CreateSamples_renewed)
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [136]:
path = r"Testing Plans\04_28_21_CdSe_Ligand_Study.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)
plan

{'Sample Amount': 5,
 'Sample Unit': 'mL',
 'Component Shorthand Names': ['CdAc', 'Se', 'OleicAc', 'Oleylamine'],
 'Component Concentration Units': ['molarity',
  'molarity',
  'molarity',
  'molarity'],
 'Component Concentration Linspaces [min, max, n]': [[0.03, 0.1, 3],
  [0.3, 0.1, 3],
  [0, 0.5, 3],
  [0, 0.25, 3]],
 'Stock Names': ['CdAc-TOP-stock',
  'Se-TOP-stock',
  'OleicAc-stock',
  'Oleylamine-stock'],
 'Stock Concentration Units': ['molarity', 'molarity', 'volf', 'volf'],
 'Stock Concentrations': [0.5, 1, 1, 1],
 'OT2 Destination Labwares': ['fischer_24_wellplate_7400ul',
  'fischer_24_wellplate_7400ul',
  'fischer_24_wellplate_7400ul',
  'fischer_24_wellplate_7400ul'],
 'OT2 Destination Labware Slots': [2, 3, 5, 6],
 'OT2 Sample Labware Start': 0,
 'OT2 Stock Labwares': ['20mlscintillation_12_wellplate_18000ul',
  '20mlscintillation_12_wellplate_18000ul'],
 'OT2 Stock Labware Slots': ['1', '4'],
 'OT2 Stock Labware Stock Order': [1, 2, 3, 4, 5],
 'OT2 Left Pipette': 'p300_

In [137]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'],
                                                                     unity_filter=False)
concentration_df

,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity
0,0.03,0.3,0.00,0.000
1,0.03,0.3,0.00,0.125
2,0.03,0.3,0.00,0.250
3,0.03,0.3,0.25,0.000
4,0.03,0.3,0.25,0.125
...,...,...,...,...
76,0.10,0.1,0.25,0.125
77,0.10,0.1,0.25,0.250
78,0.10,0.1,0.50,0.000
79,0.10,0.1,0.50,0.125


In [138]:
complete_amounts_added = CreateSamples_renewed.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
stock_dict = CreateSamples_renewed.stock_dictionary(plan['Stock Names'], plan['Stock Concentration Units'], plan['Stock Concentrations'])
complete_df = CreateSamples_renewed.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


In [139]:
complete_df = CreateSamples_renewed.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'], complete_df, stock_dict, 'Octadecene')

In [140]:
stock_volumes = CreateSamples_renewed.isolate_common_column(complete_df,'stock')
stock_volumes = CreateSamples_renewed.convert_mL_to_uL(stock_volumes)


In [71]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.3', extra_labware=custom_labware_dict) # make note this must be run constantly ~ if say making loaded dict twice
loaded_dict = ALH.loading_labware(protocol, plan)

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


In [72]:
max_source_vol = 18000 # if the list index reports out of range most likely not enough slots aka too much volume
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 
stock_position_info

{'CdAc-TOP-stock amount volume uL': {'Ranges': [[0, 28], [28, 57], [57, 81]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Se-TOP-stock amount volume uL': {'Ranges': [[0, 12],
   [12, 24],
   [24, 40],
   [40, 63],
   [63, 81]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'OleicAc-stock amount volume uL': {'Ranges': [[0, 48], [48, 81]],
  'Stock Wells': [C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   C2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Oleylamine-stock amount volume uL': {'Ranges': [[0, 81]],
  'Stock Wells': [C3 of 20mLscintillation 12 Well Plate 18000 ÂµL 

In [82]:
stock_volumes

,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,Octadecene-stock volume uL
0,300.0,1500.0,0.000000,0.000000,3200.000000
1,300.0,1500.0,0.000000,205.634994,2994.365006
2,300.0,1500.0,0.000000,411.269988,2788.730012
3,300.0,1500.0,396.713483,0.000000,2803.286517
4,300.0,1500.0,396.713483,205.634994,2597.651523
...,...,...,...,...,...
76,1000.0,500.0,396.713483,205.634994,2897.651523
77,1000.0,500.0,396.713483,411.269988,2692.016529
78,1000.0,500.0,793.426966,0.000000,2706.573034
79,1000.0,500.0,793.426966,205.634994,2500.938040


In [101]:
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict) 
# these directions should be used to make the df, unless there is a final move??



In [112]:
import pandas as pd
from pytz import timezone
import datetime

import csv

In [151]:
def add_final_location(directions, complete_df, unique_identifier= None):    
    complete_df = complete_df.copy()
    info = []
    for i, sample_info in directions.items():
        for stock, variable in sample_info.items():
            final_well_destination = variable['Destination Well Position']
        info.append(final_well_destination)    
    time = datetime.datetime.today().strftime('%m-%d-%Y') # str(datetime.datetime.now(timezone('US/Pacific')).date()) # should be embaded once you run

    wells = []
    labwares = []
    slots = []
    info_cut = info #info only being used of length of number of samples
    for info in info_cut:
        # string consist of three components, well_of_labware__on_slot with of and on being the seperators which is native and consistent across all OT2 protocols
        string = str(info)
        lower_seperator = 'of'
        upper_seperator = 'on'

        lower_seperator_index = string.index(lower_seperator)
        upper_seperator_index = string.rindex(upper_seperator)
        well = string[:lower_seperator_index-1]
        labware = string[lower_seperator_index + len(lower_seperator)+ 1:upper_seperator_index-1]
        slot = string[upper_seperator_index+len(upper_seperator)+1:]

        wells.append(well)
        labwares.append(labware)
        slots.append(slot)

    unique_identifier = None
    UIDs = []
    for slot, labware, well in zip(slots, labwares, wells):
        UID = "S" + slot + "_" + well + "_" + time  # add name of interest here to make it easier to identify
        if unique_identifier is not None: 
            UID = UID + "_" + str(unique_identifier)
        UIDs.append(UID)

    complete_df.insert(0, 'UID', UIDs)
    complete_df.insert(1, 'Labware', labwares)
    complete_df.insert(2, 'Slot', slots)
    complete_df.insert(3, 'Well', wells)
    return complete_df

In [155]:
def create_labels_for_plate():
    pass

def create_labels_for_wells():
    pass

,UID,Labware,Slot,Well,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity,CdAc amount mass g,CdAc amount volume mL,...,Se amount volume mL,OleicAc amount mass g,OleicAc amount volume mL,Oleylamine amount mass g,Oleylamine amount volume mL,CdAc-TOP-stock amount volume mL,Se-TOP-stock amount volume mL,OleicAc-stock amount volume mL,Oleylamine-stock amount volume mL,Octadecene-stock volume mL
0,S2_A1_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,2,A1,0.03,0.3,0.00,0.000,0.034575,0.014776,...,0.024624,0.000000,0.000000,0.000000,0.000000,0.3,1.5,0.000000,0.000000,3.200000
1,S2_A2_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,2,A2,0.03,0.3,0.00,0.125,0.034575,0.014776,...,0.024624,0.000000,0.000000,0.167181,0.205635,0.3,1.5,0.000000,0.205635,2.994365
2,S2_A3_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,2,A3,0.03,0.3,0.00,0.250,0.034575,0.014776,...,0.024624,0.000000,0.000000,0.334363,0.411270,0.3,1.5,0.000000,0.411270,2.788730
3,S2_A4_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,2,A4,0.03,0.3,0.25,0.000,0.034575,0.014776,...,0.024624,0.353075,0.396713,0.000000,0.000000,0.3,1.5,0.396713,0.000000,2.803287
4,S2_A5_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,2,A5,0.03,0.3,0.25,0.125,0.034575,0.014776,...,0.024624,0.353075,0.396713,0.167181,0.205635,0.3,1.5,0.396713,0.205635,2.597652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,S6_A5_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,6,A5,0.10,0.1,0.25,0.125,0.115250,0.049252,...,0.008208,0.353075,0.396713,0.167181,0.205635,1.0,0.5,0.396713,0.205635,2.897652
77,S6_A6_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,6,A6,0.10,0.1,0.25,0.250,0.115250,0.049252,...,0.008208,0.353075,0.396713,0.334363,0.411270,1.0,0.5,0.396713,0.411270,2.692017
78,S6_B1_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,6,B1,0.10,0.1,0.50,0.000,0.115250,0.049252,...,0.008208,0.706150,0.793427,0.000000,0.000000,1.0,0.5,0.793427,0.000000,2.706573
79,S6_B2_05-06-2021,Fischer 24 Well Plate 7400 ÂµL,6,B2,0.10,0.1,0.50,0.125,0.115250,0.049252,...,0.008208,0.706150,0.793427,0.167181,0.205635,1.0,0.5,0.793427,0.205635,2.500938


In [46]:
ALH.pipette_volumes_sample_wise(protocol, directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 300.0 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Aspirating 300.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 300.0 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 300.0 uL/sec
Returning tip
Dropping tip into A1 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A2 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 1500.0 from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Aspirating 750.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 750.0 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 1000.0 uL/sec
Aspirating 750.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 750.0 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 1000.0 uL/sec
Returning tip
Dropping tip 

Transferring 411.26998769987705 from C3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to D6 of Fischer 24 Well Plate 7400 ÂµL on 3
Aspirating 411.26998769987705 uL from C3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 411.26998769987705 uL into D6 of Fischer 24 Well Plate 7400 ÂµL on 3 at 1000.0 uL/sec
Returning tip
Dropping tip into A4 of Opentrons 96 Tip Rack 1000 µL on 11
Picking up tip from A5 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 2588.7300123001232 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 4 to D6 of Fischer 24 Well Plate 7400 ÂµL on 3
Aspirating 1000.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 4 at 500.0 uL/sec
Dispensing 1000.0 uL into D6 of Fischer 24 Well Plate 7400 ÂµL on 3 at 1000.0 uL/sec
Aspirating 794.3650061500616 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 4 at 500.0 uL/sec
Dispensing 794.3650061500616 uL into D6 of Fischer 24 Well Plate 7400 ÂµL on 3 at 1000.0 uL/sec
Aspirati